In [1]:
cd acme

C:\Users\Adhula\Downloads\acme


In [2]:
!pip uninstall acme

In [3]:
!pip install dm-env

In [4]:
import os

# Get the current working directory
current_directory = os.getcwd()

# Print the current working directory
print(f"Current working directory: {current_directory}")


Current working directory: C:\Users\Adhula\Downloads\acme


In [5]:
!pip install launchpad

In [6]:
# reinforcement learning
#import acme
import os
current_directory = os.getcwd()
print(f"Current working directory: {current_directory}")
print("Files in the current directory:")
for item in os.listdir(current_directory):
    if os.path.isfile(os.path.join(current_directory, item)):
        print(item)
print("Folders in the current directory:")
for item in os.listdir(current_directory):
    if os.path.isdir(os.path.join(current_directory, item)):
        print(item)
#from acme import types
from acme.types import Batches
from acme.wrappers import gym_wrapper
from acme.environment_loop import EnvironmentLoop
from acme.utils.loggers import TerminalLogger, InMemoryLogger

# environments
import gym
#import dm_env

# other
import numpy as np

Current working directory: C:\Users\Adhula\Downloads\acme
Files in the current directory:
.gitignore
.pylintrc
.readthedocs.yaml
CONTRIBUTING.md
LICENSE
MANIFEST.in
README.md
setup.py
test.sh
Folders in the current directory:
.git
.github
acme
docs
examples


## CartPole v0

In [7]:
env = gym_wrapper.GymWrapper(gym.make('CartPole-v1',new_step_api=True))
# env = acme.wrappers.SinglePrecisionWrapper(env)

# print env specs
env_specs = env.observation_space, env.action_space, env.reward_range # env.observation_spec()
print('Observation Spec:', env.observation_space)
print('Action Spec:', env.action_space)
print('Reward Spec:', env.reward_range)

Observation Spec: Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)
Action Spec: Discrete(2)
Reward Spec: (-inf, inf)


In [8]:
# show a timestep
env.reset()

TimeStep(step_type=<StepType.FIRST: 0>, reward=None, discount=None, observation=array([-0.04903064,  0.01979685,  0.01531256, -0.01966486], dtype=float32))

In [9]:
import acme

As you can see, the timesteps are somewhat different to Open AI Gym environments.

## Actor, Learners, and Agents

It is crucial to understand that there is a distinction between actors, learners, and agents. For the base class, see here: https://github.com/deepmind/acme/blob/master/acme/core.py.

In [10]:
!pip install launchpad-py

In [11]:
!pip install dm-haiku

In [12]:
!pip install dm-sonnet

In [13]:
!pip install gymnasium

In [14]:
!pip install imitation

#from acme.agents.tf import bc

# Using https://imitation.readthedocs.io/en/latest/tutorials/1_train_bc.html

In [15]:
import numpy as np
import gymnasium as gym
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper

env = make_vec_env(
    "seals:seals/CartPole-v0",
    rng=np.random.default_rng(),
    post_wrappers=[
        lambda env, _: RolloutInfoWrapper(env)
    ],  # needed for computing rollouts later
)
expert = load_policy(
    "ppo-huggingface",
    organization="HumanCompatibleAI",
    env_name="seals/CartPole-v0",
    venv=env,
)

C:\Users\Adhula\anaconda3\lib\site-packages\tensorflow\lite\python\util.py:51: DeprecationWarning: jax.xla_computation is deprecated. Please use the AOT APIs.
  from jax import xla_computation as _xla_computation


ppo-seals-CartPole-v0.zip:   0%|          | 0.00/139k [00:00<?, ?B/s]

C:\Users\Adhula\anaconda3\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Adhula\.cache\huggingface\hub\models--HumanCompatibleAI--ppo-seals-CartPole-v0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [18]:
from stable_baselines3.common.evaluation import evaluate_policy

reward, _ = evaluate_policy(expert, env, 10)
print(reward)

500.0


In [19]:
from imitation.data import rollout

rng = np.random.default_rng()
rollouts = rollout.rollout(
    expert,
    env,
    rollout.make_sample_until(min_timesteps=None, min_episodes=50),
    rng=rng,
)
transitions = rollout.flatten_trajectories(rollouts)

In [20]:
print(
    f"""The `rollout` function generated a list of {len(rollouts)} {type(rollouts[0])}.
After flattening, this list is turned into a {type(transitions)} object containing {len(transitions)} transitions.
The transitions object contains arrays for: {', '.join(transitions.__dict__.keys())}."
"""
)

The `rollout` function generated a list of 56 <class 'imitation.data.types.TrajectoryWithRew'>.
After flattening, this list is turned into a <class 'imitation.data.types.Transitions'> object containing 28000 transitions.
The transitions object contains arrays for: obs, acts, infos, next_obs, dones."



In [21]:
from imitation.algorithms import bc

bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    demonstrations=transitions,
    rng=rng,
)

In [22]:
reward_before_training, _ = evaluate_policy(bc_trainer.policy, env, 10)
print(f"Reward before training: {reward_before_training}")

Reward before training: 31.3


In [23]:
bc_trainer.train(n_epochs=1)
reward_after_training, _ = evaluate_policy(bc_trainer.policy, env, 10)
print(f"Reward after training: {reward_after_training}")

0batch [00:00, ?batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 0         |
|    ent_loss       | -0.000693 |
|    entropy        | 0.693     |
|    epoch          | 0         |
|    l2_loss        | 0         |
|    l2_norm        | 72.5      |
|    loss           | 0.692     |
|    neglogp        | 0.693     |
|    prob_true_act  | 0.5       |
|    samples_so_far | 32        |
---------------------------------


493batch [00:01, 259.47batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 500       |
|    ent_loss       | -0.000205 |
|    entropy        | 0.205     |
|    epoch          | 0         |
|    l2_loss        | 0         |
|    l2_norm        | 92.9      |
|    loss           | 0.114     |
|    neglogp        | 0.115     |
|    prob_true_act  | 0.913     |
|    samples_so_far | 16032     |
---------------------------------


866batch [00:03, 266.20batch/s]
875batch [00:03, 260.31batch/s]


Reward after training: 500.0
